Big Data Analytics Final Project Random Forest Tuning

In [1]:
from __future__ import print_function

from pyspark import SparkContext

from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils

import math


import rf_ipython
reload(rf_ipython)


<module 'rf_ipython' from 'rf_ipython.pyc'>

In [2]:
import  os
#check if pyspark env vars are set and then reset to required or delete.   
if 'PYSPARK_SUBMIT_ARGS' in os.environ:
    del os.environ['PYSPARK_SUBMIT_ARGS']

1. Tune on predicting fares (filename = 'final_fare_all.txt')

In [3]:
sc = SparkContext()
sqlContext = SQLContext(sc)
spark = SparkSession.builder.appName("RandomForestRegressor").getOrCreate()

In [5]:
performance = []
param = []

filename='final_tip_all.txt'

# Load and parse the data file into an RDD of LabeledPoint.
data = MLUtils.loadLibSVMFile(sc, filename)


test_portion=0.2

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([1-test_portion, test_portion])


cat_var_dic0 = {}
cat_var_dic0[0] = 18
#cat_var_dic0[1] = 24
cat_var_dic0[2] = 2
cat_var_dic0[3] = 3
#cat_var_dic0[4] = 10


cat_var_dic1 = {}
cat_var_dic1[0] = 18
cat_var_dic1[1] = 24
cat_var_dic1[2] = 2
cat_var_dic1[3] = 3
#cat_var_dic1[4] = 10


cat_var_dic2 = {}
cat_var_dic2[0] = 18
#cat_var_dic2[1] = 24
cat_var_dic2[2] = 2
cat_var_dic2[3] = 3
cat_var_dic2[4] = 10


cat_var_dic3 = {}
cat_var_dic3[0] = 18
cat_var_dic3[1] = 24
cat_var_dic3[2] = 2
cat_var_dic3[3] = 3
cat_var_dic3[4] = 10


iter = 0

for cat_var in [cat_var_dic0]:
    for n_tree in xrange(500, 1501, 100):
        for mode_feature_strat in ['auto', 'all', 'log2', 'onethird']:
            for max_deep in xrange(10, 21):
                for max_bin in [32, 64, 128, 256]:
                
                    iter += 1
                    print(iter)
  
    
  
    
                    param.append([cat_var, n_tree, mode_feature_strat, max_deep, max_bin])




                    #spark = SparkSession.builder.appName("RandomForestRegressor").getOrCreate()



                    ##### TREAT TEMP AS CONTINUOUS #### 
                    model = RandomForest.trainRegressor(trainingData, categoricalFeaturesInfo=cat_var,
                                                        numTrees=n_tree, featureSubsetStrategy=mode_feature_strat,
                                                        impurity='variance', maxDepth=max_deep, maxBins=max_bin)






                    ############ prediction !!!! #### 
                    # Evaluate model on test instances and compute test error
                    predictions = model.predict(testData.map(lambda x: x.features))
                    labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
                    testMSE = labelsAndPredictions.map(lambda (v, p): (v - p) * (v - p)).sum() / float(testData.count())
                    testRMSE = math.sqrt(testMSE)


                    #predictions.takeSample(withReplacement = False, num = 5)
                    # convert the rdd object to dataframe as follows
                    ##############
                    #df_predictions = predictions.map(lambda x: (x, )).toDF()
                    #df_predictions.cache()
                    ##############
                    #df_predictions.show(5, False)


                    #print('Learned regression forest model:')
                    #print(model.toDebugString())
                    print('Test Root Mean Squared Error on ' + filename + ' = ' + str(testRMSE))

                    performance.append(testRMSE)

    
best_score = min(performance)
print('minimum test error is ' + str(best_score))
index = performance.index(min(performance))
opt_param = param[index]
print('best model parameter is ' + str(opt_param))
    

1
Test Root Mean Squared Error on final_tip_all.txt = 0.25365415617
2
Test Root Mean Squared Error on final_tip_all.txt = 0.252700698118
3
Test Root Mean Squared Error on final_tip_all.txt = 0.254118123889
4
Test Root Mean Squared Error on final_tip_all.txt = 0.249854234602
5
Test Root Mean Squared Error on final_tip_all.txt = 0.253775848564
6
Test Root Mean Squared Error on final_tip_all.txt = 0.252881700863
7
Test Root Mean Squared Error on final_tip_all.txt = 0.252348263837
8
Test Root Mean Squared Error on final_tip_all.txt = 0.253112056357
9


KeyboardInterrupt: 

In [6]:
best_score = min(performance)
print('minimum test error is ' + str(best_score))
index = performance.index(min(performance))
opt_param = param[index]
print('best model parameter is ' + str(opt_param))


minimum test error is 0.249854234602
best model parameter is [{0: 18, 2: 2, 3: 3}, 500, 'auto', 10, 256]
